### ERNESTO ANDRÉS GONZÁLEZ LOMELÍ
### EXPEDIENTE: 746843

In [247]:
import numpy as np
import pandas as pd

In [248]:
data=pd.read_csv('Default.csv')
data

,default,student,balance,income
0,No,No,729.526495,44361.625074
1,No,Yes,817.180407,12106.134700
2,No,No,1073.549164,31767.138950
3,No,No,529.250605,35704.493940
4,No,No,785.655883,38463.495880
...,...,...,...,...
9995,No,No,711.555020,52992.378910
9996,No,No,757.962918,19660.721770
9997,No,No,845.411989,58636.156980
9998,No,No,1569.009053,36669.112360


In [249]:
x = data[['student', 'balance', 'income']]
y = data['default'].map({'No': 0, 'Yes': 1})


In [250]:
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

In [251]:
num_features = [ 'balance', 'income']
cat_features=['student']
num_transformer = StandardScaler()
cat_transformer= OneHotEncoder()
preprocessor = ColumnTransformer(transformers=[('num', num_transformer, num_features), 
                                                ('cat', cat_transformer, cat_features)])

In [252]:
from sklearn.tree import DecisionTreeClassifier

In [253]:
model = DecisionTreeClassifier(random_state=42)

In [254]:
from sklearn.pipeline import Pipeline

In [255]:
pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),
    ('model', model)
])

In [256]:
pipeline.fit(x, y)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['balance', 'income']),
                                                 ('cat', OneHotEncoder(),
                                                  ['student'])])),
                ('model', DecisionTreeClassifier(random_state=42))])

In [257]:
pipeline.predict(x)

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [258]:
from sklearn.model_selection import cross_val_score

In [259]:
pipeline.predict_proba(x)

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [1., 0.],
       [1., 0.],
       [1., 0.]])

In [260]:
k = 10
scores = cross_val_score(pipeline, x, y, scoring='roc_auc_ovr', cv=k)
auc = scores

In [261]:
auc_mean=np.mean(auc)
auc_mean

0.6536315476523942

In [262]:
auc_std=np.std(auc)
auc_std

0.028483442831574276

In [263]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [264]:
pipeline=Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(random_state=42))
])

In [265]:
param_grid = {
    'model__max_depth': [2,3,4],
    'model__n_estimators':[5,40]

}

In [266]:

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=10,
    n_jobs=-1,
    verbose=1
)
grid_search.fit

<bound method BaseSearchCV.fit of GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['balance',
                                                                          'income']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         ['student'])])),
                                       ('model',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'model__max_depth': [2, 3, 4],
                         'model__n_estimators': [5, 40]},
             scor

In [267]:
grid_search.fit(x, y)

Fitting 10 folds for each of 6 candidates, totalling 60 fits


GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['balance',
                                                                          'income']),
                                                                        ('cat',
                                                                         OneHotEncoder(),
                                                                         ['student'])])),
                                       ('model',
                                        RandomForestClassifier(random_state=42))]),
             n_jobs=-1,
             param_grid={'model__max_depth': [2, 3, 4],
                         'model__n_estimators': [5, 40]},
             scoring='roc_auc', verbose=1)

In [268]:
grid_search.best_params_

{'model__max_depth': 4, 'model__n_estimators': 40}

In [269]:
grid_search.best_score_

0.9371315514078151

In [270]:
best_model = grid_search.best_estimator_

In [271]:
k = 10
new_auc_scores = cross_val_score(best_model, x, y, scoring='roc_auc_ovr', cv=k)

auc_mean = np.mean(new_auc_scores)
auc_std = np.std(new_auc_scores)

In [272]:
auc_mean

0.9371315514078151

In [273]:
auc_std

0.025172128432980403

In [ ]:
#40 arboles es el mejor 
grid_search.best_params_

{'model__max_depth': 4, 'model__n_estimators': 40}